<a href="https://colab.research.google.com/github/garimagarg1999/Image_Caption_generator/blob/main/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests
!pip install beautifulsoup4


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

def get_product_data(product_url):
    response = requests.get(product_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    product_name_element = soup.find('span', {'id': 'productTitle'})
    product_name = product_name_element.get_text().strip() if product_name_element else ''

    product_price_element = soup.find('span', {'id': 'priceblock_ourprice'})
    product_price = product_price_element.get_text().strip() if product_price_element else ''

    rating_element = soup.find('span', {'class': 'a-icon-alt'})
    rating = rating_element.get_text().split()[0] if rating_element else ''

    num_reviews_element = soup.find('span', {'id': 'acrCustomerReviewText'})
    num_reviews = num_reviews_element.get_text().split()[0] if num_reviews_element else ''

    description_element = soup.find('div', {'id': 'productDescription'})
    description = description_element.get_text().strip() if description_element else ''

    asin_element = soup.find('th', string='ASIN')
    asin = asin_element.find_next('td').get_text().strip() if asin_element else ''

    manufacturer_element = soup.find('th', string='Manufacturer')
    manufacturer = manufacturer_element.find_next('td').get_text().strip() if manufacturer_element else ''

    return [product_name, product_price, rating, num_reviews, description, asin, manufacturer]


def scrape_product_listings(url, num_pages):
    all_products = []

    for page in range(1, num_pages+1):
        page_url = url + f'&page={page}'
        response = requests.get(page_url)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all the product cards on the page
        product_cards = soup.find_all('div', {'data-component-type': 's-search-result'})

        for card in product_cards:
            product_url = 'https://www.amazon.in' + card.find('a', {'class': 'a-link-normal'})['href']
            product_data = get_product_data(product_url)
            product_data.insert(0, product_url)  # Add the product URL at the beginning
            all_products.append(product_data)

    return all_products

# Scrape product listings
base_url = 'https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_'
num_pages = 20
product_listings = scrape_product_listings(base_url, num_pages)

# Export data to CSV file
csv_filename = 'product_data.csv'
header = ['Product URL', 'Product Name', 'Product Price', 'Rating', 'Number of Reviews',
          'Description', 'ASIN', 'Manufacturer']

with open(csv_filename, 'w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(product_listings)


In [ ]:
import time

def scrape_product_urls(urls):
    product_data = []

    for url in urls:
        data = get_product_data(url)
        product_data.append([url] + data)
        time.sleep(1)  # Add a delay to avoid overwhelming the server

    return product_data

# Extract product URLs from the scraped product listings
product_urls = [product[0] for product in product_listings]

# Scrape additional information for each product URL
product_data = scrape_product_urls(product_urls[:200])  # Limit to 200 URLs

# Export updated data to CSV file
csv_filename = 'product_data_updated.csv'
header = ['Product URL', 'Product Name', 'Product Price', 'Rating', 'Number of Reviews',
          'Description', 'ASIN', 'Manufacturer']

with open(csv_filename, 'w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(product_data)


In [ ]:
from google.colab import files

# Download the CSV file
files.download('product_data_updated.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>